In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

## IO markup

### Train

In [2]:
from modules.data import bert_data

In [3]:
train_df_path = "/home/eartemov/ae/work/factRuEval-2016/dev.csv"
valid_df_path = "/home/eartemov/ae/work/factRuEval-2016/test.csv"

In [5]:
train_df_path = "/home/eartemov/ae/work/factRuEval-2016/dev.csv"
valid_df_path = "/home/eartemov/ae/work/factRuEval-2016/test.csv"
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels4.txt",
    clear_cache=True
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [6]:
from modules.models.bert_models import BERTBiLSTMCRF

In [7]:
model = BERTBiLSTMCRF.create(len(data.train_ds.idx2label), lstm_dropout=0., crf_dropout=0.3)

In [8]:
from modules.train.train import NerLearner

In [9]:
num_epochs = 100

In [12]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTBiLSTMCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [13]:
model.get_n_trainable_params()

2234745

In [ ]:
learner.fit(epochs=num_epochs)

### Predict

In [14]:
from modules.data.bert_data import get_data_loader_for_predict

In [15]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [16]:
preds = learner.predict(dl)

In [17]:
from sklearn_crfsuite.metrics import flat_classification_report

In [18]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [19]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [20]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=["I_ORG", "I_PER", "I_LOC"], digits=4)

In [21]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.8579    0.7917    0.8235      3865
       I_PER     0.9510    0.9659    0.9584      2112
       I_LOC     0.9053    0.8349    0.8687      1557

   micro avg     0.8954    0.8495    0.8718      7534
   macro avg     0.9047    0.8642    0.8835      7534
weighted avg     0.8938    0.8495    0.8706      7534



In [26]:
from modules.analyze_utils.main_metrics import precision_recall_f1

In [27]:
results = precision_recall_f1(true_labels, pred_labels)

processed 56409 tokens with 7534 phrases; found: 7148 phrases; correct: 6400.

precision:  89.54%; recall:  84.95%; FB1:  87.18

	LOC: precision:  90.53%; recall:  83.49%; F1:  86.87 1436

	ORG: precision:  85.79%; recall:  79.17%; F1:  82.35 3567

	PER: precision:  95.10%; recall:  96.59%; F1:  95.84 2145


